# 케라스로 VGGNet 만들기

이 노트북에서 "매우 깊은" 합성곱 신경망 [VGGNet](https://arxiv.org/pdf/1409.1556.pdf)과 비슷한 모델을 훈련하여 옥스포드 꽃 데이터셋을 17개 카테고리로 분류합니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/dl-illustrated/blob/master/notebooks/10-3.vggnet_in_keras.ipynb)

#### 동일하게 재현하기 위해 랜덤 시드 값을 지정합니다.

In [1]:
import numpy as np
np.random.seed(42)

#### 라이브러리를 적재합니다.

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard 

#### 데이터를 적재하고 전처리합니다.

이미 `oxflower17.npz` 파일을 다운받았다고 가정합니다. 이 파일이 없다면 `10-2.alexnet_in_keras.ipynb` 노트북을 다시 실행하세요.

In [3]:
import numpy as np

data = np.load('oxflower17.npz')
X = data['X']
Y = data['Y']

#### 신경망 모델을 만듭니다.

In [4]:
model = Sequential()

model.add(Conv2D(64, 3, activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(128, 3, activation='relu'))
model.add(Conv2D(128, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(256, 3, activation='relu'))
model.add(Conv2D(256, 3, activation='relu'))
model.add(Conv2D(256, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(17, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 110, 110, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 128)       0

#### 모델을 설정합니다.

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 텐서보드를 설정합니다.

In [7]:
tensorbrd = TensorBoard('logs/vggnet')

#### 훈련!

In [8]:
model.fit(X, Y, batch_size=64, epochs=250, verbose=1, validation_split=0.1, shuffle=True, callbacks=[tensorbrd])

Epoch 1/250
 1/20 [>.............................] - ETA: 0s - loss: 3.1597 - accuracy: 0.0312WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
20/20 [==============================] - 7s 363ms/step - loss: 3.3338 - accuracy: 0.1258 - val_loss: 21.3062 - val_accuracy: 0.0662
Epoch 2/250
20/20 [==============================] - 5s 233ms/step - loss: 2.6855 - accuracy: 0.1789 - val_loss: 17.5411 - val_accuracy: 0.0368
Epoch 3/250
20/20 [==============================] - 5s 230ms/step - loss: 2.5812 - accuracy: 0.2345 - val_loss: 8.9391 - val_accuracy: 0.0000e+00
Epoch 4/250
20/20 [==============================] - 5s 233ms/step - loss: 2.2845 - accuracy: 0.2639 - val_loss: 6.5614 - val_accuracy: 0.0588
Epoch 5/250
20/20 [==============================] - 5s 233ms/